In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


import re
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx


import nltk
import string
import re
from nltk.corpus import stopwords
import networkx as nx
from stemming.porter2 import stem
import csv
from networkx.algorithms import community

df_userdata = pd.read_csv(input("Transaction Data:"),sep="\t")
df_userdata.columns = ['source','target']

SNAP = open('D:/Bajaj/amazon-meta.txt/amazon-meta.txt', 'r', encoding='utf-8', errors='ignore')
SNAP_dict = {}
(Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating, DegreeCentrality, ClusteringCoeff) = ("", "", "", "", "", "", 0, 0, 0.0, 0, 0.0)

for line in SNAP:
    line = line.strip()

    if(line.startswith("Id")):
        Id = line[3:].strip()
        
    elif(line.startswith("ASIN")):
        ASIN = line[5:].strip()
        
    elif(line.startswith("title")):
        Title = line[6:].strip()
        Title = ' '.join(Title.split())
        
    elif(line.startswith("group")):
        Group = line[6:].strip()
        
    elif(line.startswith("salesrank")):
        SalesRank = line[10:].strip()
        
    elif(line.startswith("similar")):
        Itemset = line.split()
        Copurchased = ' '.join([character for character in Itemset[2:]])
        
    elif(line.startswith("categories")):
        Itemset = line.split()
        Categories = ' '.join((SNAP.readline()).lower() for i in range(int(Itemset[1].strip())))
        Categories = re.compile('[%s]' % re.escape(string.digits+string.punctuation)).sub(' ', Categories)
        Categories = ' '.join(set(Categories.split())-set(stopwords.words("english")))        
        Categories = ' '.join(stem(word) for word in Categories.split())
        
    elif(line.startswith("reviews")):
        ls = line.split()
        TotalReviews = ls[2].strip()
        AvgRating = ls[7].strip()
        

    elif (line==""):
        try:
            MetaData = {}
            if (Id != ""):
                SNAP_dict[Id]=MetaData
            MetaData['ASIN'] = ASIN            
            MetaData['Title'] = Title
            MetaData['Categories'] = ' '.join(set(Categories.split()))
            MetaData['Group'] = Group
            MetaData['Copurchased'] = Copurchased
            MetaData['SalesRank'] = int(SalesRank)
            MetaData['TotalReviews'] = int(TotalReviews)
            MetaData['AvgRating'] = float(AvgRating)
            MetaData['DegreeCentrality'] = DegreeCentrality
            MetaData['ClusteringCoeff'] = ClusteringCoeff
        except NameError:
            continue
        (Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating, DegreeCentrality, ClusteringCoeff) = ("", "", "", "", "", "", 0, 0, 0.0, 0, 0.0)
SNAP.close()

items = list(SNAP_dict.items())
ls = []
for i, j in enumerate(items):
    colnames = list(j[1])
    colnames.append("ASIN")
    vals = list(j[1].values())
    vals.append(j[0])
    ls.append(vals)
df = pd.DataFrame(ls, columns=colnames)


n = int(input("Input the number of transactions to be considered in the subgraph"))
df_small = df_userdata[:n]

G_small = nx.DiGraph()
G_small = nx.from_pandas_edgelist(df_small,source='source',target='target')

communities_generator = community.girvan_newman(G_small)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)

top_level_communities = top_level_communities[0]

print("Top Recommendations:")
print(".........................................")
result = []
for i in top_level_communities:
    result.append(df.iloc[i])
print('\n \n \n'.join(map(str, result)))


Transaction Data:D:/Bajaj/amazon0601.txt/Amazon0601.txt
Input the number of transactions to be considered in the subgraph100
Top Recommendations:
.........................................
ASIN                                                       0827229534
Title                         Patterns of Preaching: A Sermon Sampler
Categories          religion sermon christian book preach subject ...
Group                                                            Book
Copurchased         0804215715 156101074X 0687023955 0687074231 08...
SalesRank                                                      396585
TotalReviews                                                        2
AvgRating                                                           5
DegreeCentrality                                                    0
ClusteringCoeff                                                     0
ASIN                                                                1
Name: 0, dtype: object
 
 
ASIN           